# Pricing a Call Option with the Binomial Model

In [179]:
using CSV
using DelimitedFiles, DataFrames
using Statistics
import PyPlot as plt
using Plots
using GLM
using ShiftedArrays
using PyCall
using PlotlyJS

In [180]:
GOOG         = DataFrame(CSV.File("DATA/GOOG.csv"))
BNTX         = DataFrame(CSV.File("DATA/BNTX.csv"))
BNTX_6MONTHS = DataFrame(CSV.File("DATA/BNTX_6MONTHS.csv"))

,Date,Open,High,Low,Close,Adj Close,Volume
,Date,Float64,Float64,Float64,Float64,Float64,Int64
1,2021-09-20,341.205,353.64,333.36,339.01,339.01,3322100
2,2021-09-21,344.77,349.93,337.04,341.35,341.35,2455400
3,2021-09-22,342.803,344.57,336.69,339.26,339.26,1480000
4,2021-09-23,343.0,360.0,341.44,352.99,352.99,2824400
5,2021-09-24,350.0,350.92,332.4,333.48,333.48,2603100
6,2021-09-27,335.361,338.5,305.51,307.09,307.09,4089300
7,2021-09-28,293.7,296.9,275.0,276.52,276.52,5854400
8,2021-09-29,287.0,292.81,275.58,277.44,277.44,2776300
9,2021-09-30,278.14,282.5,268.83,272.99,272.99,2632600


In [181]:
p = PlotlyJS.plot(candlestick(x    =GOOG[!, "Date" ],
                 open =GOOG[!, "Open" ],
                 high =GOOG[!, "High" ],
                 low  =GOOG[!, "Low"  ],
                 close=GOOG[!, "Close"] ),
    Layout(title="GOOG Prices",
           yaxis_title="GOOG Stock")
    )

data: [
  "candlestick with fields close, high, low, open, type, and x"
]

layout: "layout with fields margin, template, title, and yaxis"

In [182]:
PlotlyJS.savefig(p, "figures/GOOG.html")

"figures/GOOG.html"

In [183]:
p = PlotlyJS.plot(candlestick(
        x    =BNTX[!, "Date" ],
        open =BNTX[!, "Open" ],
        high =BNTX[!, "High" ],
        low  =BNTX[!, "Low"  ],
        close=BNTX[!, "Close"] ),
    Layout(title="BioNTech Prices",
           yaxis_title="BNTX Stock")
    )

data: [
  "candlestick with fields close, high, low, open, type, and x"
]

layout: "layout with fields margin, template, title, and yaxis"

In [184]:
PlotlyJS.savefig(p, "figures/BNTX.html")

"figures/BNTX.html"

In [280]:
p = PlotlyJS.plot(candlestick(
        x    =BNTX_6MONTHS[!, "Date" ],
        open =BNTX_6MONTHS[!, "Open" ],
        high =BNTX_6MONTHS[!, "High" ],
        low  =BNTX_6MONTHS[!, "Low"  ],
        close=BNTX_6MONTHS[!, "Close"] ),
    Layout(title="BioNTech Prices",
           yaxis_title="BNTX Stock [\$]")
    )

data: [
  "candlestick with fields close, high, low, open, type, and x"
]

layout: "layout with fields margin, template, title, and yaxis"

In [186]:
PlotlyJS.savefig(p, "figures/BNTX_6MONTHS.html")

"figures/BNTX_6MONTHS.html"

In [187]:
function Return( df )
    df[!,:Return    ] =     ( df.Close - lag(df.Close, 1) ) ./ lag(df.Close, 1)
    df[!,:Return_LOG] = log.( df.Close                      ./ lag(df.Close, 1) )
end

Return( GOOG         )
Return( BNTX         )
Return( BNTX_6MONTHS )
GOOG

,Date,Open,High,Low,Close,Adj Close,Volume,Return
,Date,Float64,Float64,Float64,Float64,Float64,Int64,Float64?
1,2021-12-13,2968.88,2971.25,2927.2,2934.09,2934.09,1205200,missing
2,2021-12-14,2895.4,2908.84,2844.85,2899.41,2899.41,1238900,-0.0118197
3,2021-12-15,2887.32,2950.34,2854.11,2947.37,2947.37,1364000,0.0165414
4,2021-12-16,2961.54,2971.03,2881.85,2896.77,2896.77,1370000,-0.0171679
5,2021-12-17,2854.29,2889.2,2835.76,2856.06,2856.06,2162800,-0.0140536
6,2021-12-20,2813.59,2852.21,2805.0,2848.03,2848.03,1013200,-0.00281158
7,2021-12-21,2863.0,2893.84,2834.7,2884.41,2884.41,977400,0.0127737
8,2021-12-22,2882.0,2946.06,2879.26,2938.98,2938.98,921900,0.018919
9,2021-12-23,2941.79,2971.45,2939.02,2942.85,2942.85,690900,0.00131682


In [188]:
PlotlyJS.plot( GOOG.Date, GOOG.Return,mode="markers+lines",
               Layout(title=      "GOOG Returns",
                      yaxis_title="GOOG Returns") )

data: [
  "scatter with fields mode, type, x, and y"
]

layout: "layout with fields margin, template, title, and yaxis"

In [189]:
PlotlyJS.plot( BNTX.Date, BNTX.Return,mode="markers+lines",
               Layout(title="BioNTech Returns",
                      yaxis_title="BNTX Returns") )

data: [
  "scatter with fields mode, type, x, and y"
]

layout: "layout with fields margin, template, title, and yaxis"

In [190]:
PlotlyJS.plot( BNTX_6MONTHS.Date, BNTX_6MONTHS.Return,mode="markers+lines", name="Returns",
               Layout(title="BioNTech Returns",
                      yaxis_title="BNTX Returns") )

data: [
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, and yaxis"

## 1-Step Binomial Tree

In [191]:
Year_Days               = 252
Interest_rate3          = 0.0080286 # 11/03/22
Interest_rate6          = 0.0128757 # 20/03/22
Maturity3               = 3/12
Maturity6               = 6/12
S0_GOOG                 = GOOG.Close[length(GOOG.Close)]
S0_BNTX                 = BNTX.Close[length(BNTX.Close)]
S0_BNTX_6MONTHS         = BNTX.Close[length(BNTX.Close)]
K_GOOG                  = 2600
K_BNTX                  = 140
K_BNTX_6MONTHS          = 175

VOL_DAILY_GOOG          = std(skipmissing(GOOG.Return))
VOL_ANNUAL_GOOG         = VOL_DAILY_GOOG * sqrt(Year_Days)

VOL_DAILY_BNTX          = std(skipmissing(BNTX.Return))
VOL_ANNUAL_BNTX         = VOL_DAILY_BNTX * sqrt(Year_Days)

VOL_DAILY_BNTX_6MONTHS  = std(skipmissing(BNTX_6MONTHS.Return))
VOL_ANNUAL_BNTX_6MONTHS = VOL_DAILY_BNTX_6MONTHS * sqrt(Year_Days)

println("Daily volatility of GOOG in the last 3 month:\t", VOL_DAILY_GOOG         )
println("Daily volatility of BNTX in the last 3 month:\t", VOL_DAILY_BNTX         )
println("Daily volatility of BNTX in the last 6 month:\t", VOL_DAILY_BNTX_6MONTHS )
println("\n")

println("Annual volatility of GOOG in the last 3 month:\t", VOL_ANNUAL_GOOG         )
println("Annual volatility of BNTX in the last 3 month:\t", VOL_ANNUAL_BNTX         )
println("Annual volatility of BNTX in the last 6 month:\t", VOL_ANNUAL_BNTX_6MONTHS )
println("\n")

UpDown_GOOG             = ( exp( + VOL_ANNUAL_GOOG         * sqrt( Maturity3 ) ), exp( - VOL_ANNUAL_GOOG         * sqrt( Maturity3 ) ) )
UpDown_BNTX             = ( exp( + VOL_ANNUAL_BNTX         * sqrt( Maturity3 ) ), exp( - VOL_ANNUAL_BNTX         * sqrt( Maturity3 ) ) )
UpDown_BNTX_6MONTHS     = ( exp( + VOL_ANNUAL_BNTX_6MONTHS * sqrt( Maturity6 ) ), exp( - VOL_ANNUAL_BNTX_6MONTHS * sqrt( Maturity6 ) ) )

q_GOOG                  = ( ( 1 + Maturity3 * Interest_rate3 ) - UpDown_GOOG[2] )         / ( UpDown_GOOG[1]         - UpDown_GOOG[2]         )
q_BNTX                  = ( ( 1 + Maturity3 * Interest_rate3 ) - UpDown_BNTX[2] )         / ( UpDown_BNTX[1]         - UpDown_BNTX[2]         )
q_BNTX_6MONTHS          = ( ( 1 + Maturity6 * Interest_rate6 ) - UpDown_BNTX_6MONTHS[2] ) / ( UpDown_BNTX_6MONTHS[1] - UpDown_BNTX_6MONTHS[2] )

PRICE_CALL_GOOG         = (     q_GOOG           * max( 0, S0_GOOG         * UpDown_GOOG[1]         - K_GOOG )           + 
                          ( 1 - q_GOOG         ) * max( 0, S0_GOOG         * UpDown_GOOG[2]         - K_GOOG ) )         / ( 1 + Maturity3 * Interest_rate6 )
PRICE_CALL_BNTX         = (     q_BNTX           * max( 0, S0_BNTX         * UpDown_BNTX[1]         - K_BNTX )           + 
                          ( 1 - q_BNTX         ) * max( 0, S0_BNTX         * UpDown_BNTX[2]         - K_BNTX ) )         / ( 1 + Maturity3 * Interest_rate6 )
PRICE_CALL_BNTX_6MONTHS = (     q_BNTX_6MONTHS   * max( 0, S0_BNTX_6MONTHS * UpDown_BNTX_6MONTHS[1] - K_BNTX_6MONTHS )   + 
                          ( 1 - q_BNTX_6MONTHS ) * max( 0, S0_BNTX_6MONTHS * UpDown_BNTX_6MONTHS[2] - K_BNTX_6MONTHS ) ) / ( 1 + Maturity6 * Interest_rate6 )

println("Price of a Call Option of GOOGLE   with 3 month Maturity:\t", PRICE_CALL_GOOG         )
println("Price of a Call Option of BioNTech with 3 month Maturity:\t", PRICE_CALL_BNTX         )
println("Price of a Call Option of BioNTech with 6 month Maturity:\t", PRICE_CALL_BNTX_6MONTHS )

Daily volatility of GOOG in the last 3 month:	0.02138303844467155
Daily volatility of BNTX in the last 3 month:	0.0490511345265535
Daily volatility of BNTX in the last 6 month:	0.05745107644746378


Annual volatility of GOOG in the last 3 month:	0.3394452119972058
Annual volatility of BNTX in the last 3 month:	0.7786626208970072
Annual volatility of BNTX in the last 6 month:	0.9120075649976959


Price of a Call Option of GOOGLE   with 3 month Maturity:	251.5961127010554
Price of a Call Option of BioNTech with 3 month Maturity:	25.36241269490067
Price of a Call Option of BioNTech with 6 month Maturity:	30.01503784429265


### Multisteps-Binomial Tree

In [192]:
function CALL_PRICE(S, K, r, σ, T, N)
    Δt     = T / N
    U      = exp(σ * √Δt)
    D      = 1 / U
    R      = ( 1 + r * Δt )
    q      = (R - D) / (U - D)

    Payoff = [max(0, S * U^(N-i) * D^i - K ) for i = 0:N]
    
    return sum( Payoff[j+1] * binomial(N,j) * (1-q)^j * (q)^(N-j) for j in 0:N ) / R
end

CALL_PRICE (generic function with 1 method)

In [193]:
print( "\n", CALL_PRICE(S0_GOOG,         K_GOOG,         Interest_rate3, VOL_ANNUAL_GOOG,         Maturity3, 30) )
print( "\n", CALL_PRICE(S0_BNTX,         K_BNTX,         Interest_rate3, VOL_ANNUAL_BNTX,         Maturity3, 30) )
print( "\n", CALL_PRICE(S0_BNTX_6MONTHS, K_BNTX_6MONTHS, Interest_rate6, VOL_ANNUAL_BNTX_6MONTHS, Maturity6, 60) )


209.95074291428583
20.201281731270587
23.831058568336893

In [194]:
GOOG_CALL         = [CALL_PRICE(S0_GOOG,         K_GOOG,         Interest_rate3, VOL_ANNUAL_GOOG,         Maturity3, i) for i in 1:60]
BNTX_CALL         = [CALL_PRICE(S0_BNTX,         K_BNTX,         Interest_rate3, VOL_ANNUAL_BNTX,         Maturity3, i) for i in 1:60]
BNTX_CALL_6MONTHS = [CALL_PRICE(S0_BNTX_6MONTHS, K_BNTX_6MONTHS, Interest_rate6, VOL_ANNUAL_BNTX_6MONTHS, Maturity6, i) for i in 1:60]
print("OK")

OK

In [195]:
REAL_CALL_GOOG         = 209.6
REAL_CALL_BNTX         =  21.7
REAL_CALL_BNTX_6MONTHS =  14.0 #30.65

14.0

In [196]:
function PlotGOOG_3MONTHS()
    GOOG_PLOT = PlotlyJS.scatter(;x=1:60, y=GOOG_CALL,mode="markers+lines", name="Binomial Model" )
    trace     = PlotlyJS.scatter(;x=1:60, y=repeat([REAL_CALL_GOOG], 60), mode="lines", name="Real Value", line=attr(color="firebrick", width=4, dash="dashdot"))
    layout    = Layout(title="Google Call Price, Maturity 3 months, Strike ~ S0, as function of number of steps",
                       yaxis_title="CALL price [\$]", xaxis_title="Steps")
    return PlotlyJS.plot([GOOG_PLOT,trace], layout)
end

p = PlotGOOG_3MONTHS()
    

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

In [197]:
PlotlyJS.savefig(p, "figures/CALL_GOOG.pdf")

"figures/CALL_GOOG.pdf"

In [198]:
function PlotBNTX_3MONTHS()
    BNTX_PLOT = PlotlyJS.scatter(;x=1:60, y=BNTX_CALL,mode="markers+lines", name="Binomial Model" )
    trace     = PlotlyJS.scatter(;x=1:60, y=repeat([REAL_CALL_BNTX], 60), mode="lines", name="Real Value", line=attr(color="firebrick", width=4, dash="dashdot"))
    layout    = Layout(title="BioNTech Call Price, Maturity 3 months, Strike ~ S0, as function of number of steps",
                       yaxis_title="CALL price [\$]", xaxis_title="Steps")
    return PlotlyJS.plot([BNTX_PLOT,trace], layout)
end

p3 = PlotBNTX_3MONTHS()

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

In [199]:
PlotlyJS.savefig(p3, "figures/CALL_BNTX.png")

"figures/CALL_BNTX.png"

In [200]:
function PlotBNTX_6MONTHS()
    BNTX_PLOT_6MONTHS = PlotlyJS.scatter(;x=1:60, y=BNTX_CALL_6MONTHS,mode="markers+lines", name="Binomial Model" )
    trace     = PlotlyJS.scatter(;x=1:60, y=repeat([REAL_CALL_BNTX_6MONTHS], 60), mode="lines", name="Real Value", line=attr(color="firebrick", width=4, dash="dashdot"))
    layout    = Layout(title="BioNTech Call Price, Maturity 6 months, Strike ~ S0, as function of number of steps",
                       yaxis_title="CALL price [\$]", xaxis_title="Steps")
    return PlotlyJS.plot([BNTX_PLOT_6MONTHS,trace], layout)
end

p6 = PlotBNTX_6MONTHS()

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields line, mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

In [201]:
PlotlyJS.savefig(p6, "figures/CALL_BNTX_6MONTHS.pdf")

"figures/CALL_BNTX_6MONTHS.pdf"

In [235]:
function PlotBNTX_6MONTHS()
    BNTX_PLOT_6MONTHS = PlotlyJS.scatter(;x=1:60, y=BNTX_CALL_6MONTHS,mode="markers+lines", name="Binomial Model" )
    trace     = PlotlyJS.scatter(;x=1:60, y=repeat([REAL_CALL_BNTX_6MONTHS], 60), mode="lines", name="Real Value", line=attr(color="firebrick", width=4, dash="dashdot"))
    layout    = Layout(title="Maturity 6 months",
                       yaxis_title="CALL price [\$]", xaxis_title="Steps")
    return PlotlyJS.plot([BNTX_PLOT_6MONTHS,trace], layout)
end
function PlotBNTX_3MONTHS()
    BNTX_PLOT = PlotlyJS.scatter(;x=1:60, y=BNTX_CALL,mode="markers+lines", name="Binomial Model" )
    trace     = PlotlyJS.scatter(;x=1:60, y=repeat([REAL_CALL_BNTX], 60), mode="lines", name="Real Value", line=attr(color="firebrick", width=4, dash="dashdot"))
    layout    = Layout(title="Maturity 3 months",
                       yaxis_title="CALL price [\$]")
    return PlotlyJS.plot([BNTX_PLOT,trace], layout)
end

p = [PlotBNTX_3MONTHS(); PlotBNTX_6MONTHS()]

relayout!(p,height=600,title_text="BioNTech Call Price, Strike ~ S0, as function of number of steps", )

p

data: [
  "scatter with fields mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, height, margin, template, title, xaxis1, xaxis2, yaxis1, and yaxis2"

In [279]:
p = PlotlyJS.make_subplots(rows=2, cols=1, shared_xaxes=true, vertical_spacing=0.08, subplot_titles=["3 Months maturity" "6 Months maturity"], y_title="CALL price [\$]", x_title="Steps")
PlotlyJS.add_trace!(p, PlotlyJS.scatter(;x=1:60, y=BNTX_CALL_6MONTHS,mode="markers+lines", name="Binomial Model 6M" ), row=2, col=1)
PlotlyJS.add_trace!(p, PlotlyJS.scatter(;x=1:60, y=repeat([REAL_CALL_BNTX_6MONTHS], 60), mode="lines", name="Real Value 6M", line=attr(color="firebrick", width=4, dash="dashdot")), row=2, col=1)


PlotlyJS.add_trace!(p, PlotlyJS.scatter(;x=1:60, y=BNTX_CALL,mode="markers+lines", name="Binomial Model 3M" ), row=1, col=1)
PlotlyJS.add_trace!(p, PlotlyJS.scatter(;x=1:60, y=repeat([REAL_CALL_BNTX], 60), mode="lines", name="Real Value 3M", line=attr(color="orange", width=4, dash="dashdot")), row=1, col=1)

p.plot.layout.annotations[3].yshift=-15
p.plot.layout.annotations[4].xshift=-18
PlotlyJS.relayout!(p, title_text="BioNTech Call Price, Strike ~ S0, as function of number of steps")
#print(json(p.plot.layout.annotations, 2))
p


data: [
  "scatter with fields mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields mode, name, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, margin, template, title, xaxis, xaxis2, yaxis, and yaxis2"

In [154]:
CALL_PRICES_BNTX = DataFrame(CSV.File("DATA/CALL_BNTX_20032022.csv"))

,Contract Name,Last Trade Date,Strike,Last Price,Bid,Ask
,String,String,Float64,Float64,Float64,Float64
1,BNTX220916C00100000,20220318 10:34AM EDT,100.0,79.8,75.8,85.0
2,BNTX220916C00110000,20220124 10:33AM EDT,110.0,45.0,49.9,57.3
3,BNTX220916C00130000,20220315 1:08PM EDT,130.0,38.0,53.2,62.5
4,BNTX220916C00140000,20220223 1:52PM EDT,140.0,30.65,46.8,56.2
5,BNTX220916C00145000,20220223 11:42AM EDT,145.0,30.57,44.3,53.3
6,BNTX220916C00150000,20220318 1:40PM EDT,150.0,44.75,41.4,50.0
7,BNTX220916C00155000,20220318 2:04PM EDT,155.0,40.94,38.8,47.3
8,BNTX220916C00160000,20220318 3:41PM EDT,160.0,39.44,36.2,44.6
9,BNTX220916C00165000,20220317 9:30AM EDT,165.0,35.7,33.9,41.4


In [158]:
PlotlyJS.plot( CALL_PRICES_BNTX.Strike, CALL_PRICES_BNTX[!, Symbol("Volume")] ,mode="markers+lines",
               Layout(title="BioNTech Call option volume as function of the strike",
                      yaxis_title="Volume", xaxis_title="Strike [\$]") )

data: [
  "scatter with fields mode, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

In [111]:
BNTX_CALL_6MONTHS_STRIKES = [CALL_PRICE(S0_BNTX_6MONTHS, CALL_PRICES_BNTX.Strike[i], Interest_rate6, VOL_ANNUAL_BNTX_6MONTHS, Maturity6, 60) for i in 1:length(CALL_PRICES_BNTX.Strike)]
print("OK")

OK

In [175]:
a = PlotlyJS.scatter( ;x=CALL_PRICES_BNTX.Strike, y=BNTX_CALL_6MONTHS_STRIKES,                 mode="markers+lines", name="60-steps binomial tree prediction")
b = PlotlyJS.scatter( ;x=CALL_PRICES_BNTX.Strike, y=CALL_PRICES_BNTX[!, Symbol("Last Price")], mode="markers+lines", name="Real price of the option")
layout    = Layout(title="BioNTech Call option prices", yaxis_title="Call prices [\$]", xaxis_title="Strike prices [\$]")
PlotlyJS.plot([a,b], layout)

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

In [262]:
@doc PlotlyJS.relayout

No documentation found.

`PlotlyBase.relayout` is a `Function`.

```
# 2 methods for generic function "relayout":
[1] relayout(p::Plot, args...; kwargs...) in PlotlyBase at /home/aidin/.julia/packages/PlotlyBase/xb3Du/src/api.jl:458
[2] relayout(plt::PlotlyJS.SyncPlot, args...; kwargs...) in PlotlyJS at /home/aidin/.julia/packages/PlotlyJS/4jzLr/src/display.jl:315
```
